# BASIN moveout plot notebooks
This notebook makes moveout plots of the stacked cross correlations between broadbands and nodal stations. It is highly inspired by Julian Schmitt's work, modified by Marine Denolle (mdenolle@uw.edu)

In [1]:
# Packages - set GR as plotting backend 
#include("/Users/julianschmitt/Documents/Schoolwork/Seismology/SeisCore.jl/src/SeisCore.jl")
using Pkg,  HDF5, SeisIO, SeisNoise, Plots, Dates, CSV, DataFrames, Distributed, JLD2, Statistics, PyCall, Glob, StructArrays, ColorSchemes, Plots.PlotUtils, AbstractFFTs
# loadcolorscheme(:cm_maxamp,ColorSchemes.gist_heat.colors[end-50:-1:1], "maxamp color", "for waveform plot");

┌ Warning: Error requiring `HDF5` from `Plots`
│   exception = (LoadError("/Users/marinedenolle/.julia/packages/Plots/zozYv/src/backends/hdf5.jl", 36, UndefVarError(:Group)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x000000010fc67b1f, Ptr{Nothing} @0x000000010fd113cf, Ptr{Nothing} @0x000000010fd1257b, Ptr{Nothing} @0x000000010fd10ae6, Ptr{Nothing} @0x000000010fd110dc, Base.InterpreterIP in top-level CodeInfo for Plots._hdf5_implementation at statement 4, Ptr{Nothing} @0x000000010fd2c15f, Ptr{Nothing} @0x000000010fd2c2f5, Ptr{Nothing} @0x000000010fd2bf02, Ptr{Nothing} @0x000000010fd2bf02, Ptr{Nothing} @0x000000010fd2d1d5, Ptr{Nothing} @0x000000011f5cc7da, Ptr{Nothing} @0x000000010fcf9160, Ptr{Nothing} @0x000000011f60bf45, Ptr{Nothing} @0x000000011f197c85, Ptr{Nothing} @0x000000010fcf9160, Ptr{Nothing} @0x000000010fd05f4a, Ptr{Nothing} @0x000000011d33a71a, Ptr{Nothing} @0x000000010fcf9160, Ptr{Nothing} @0x000000010fd1259f, Ptr{Nothing} @0x000000010fd10ae6, Ptr{Nothing} @0x0

In [2]:
include("../../src/process.jl")
include("/Users/marinedenolle/BASIN/C4-Project.jl/src/process.jl")
include("../src/utils.jl")
ENV["GR"] = ""
# Pkg.build("GR")

""

## function to plot per distance range

In [14]:
function plot_by_latitude(corrs::Array{CorrData,1}, freqs::Array{Array{Float64,1},1}, 
                            attr::Array{String,1}=["","","","",""], src_geo::Any=nothing, fs::Float64=20.)
    println("crap!")
#     loadcolorscheme(:cm_maxamp,ColorSchemes.hot.colors[end-50:-1:1], "maxamp color", "for waveform plot");
#     loadcolorscheme(:cm_maxamp,ColorSchemes.jet1.colors[end-50:-1:1], "maxamp color", "for waveform plot");
    loadcolorscheme(:cm_maxamp,ColorSchemes.jet1.colors[end:-1:1], "maxamp color", "for waveform plot");
    source = attr[1]
    println(source)
    println("Sorting by distance")
    plot_sorttype == "Latitude"
    println(corrs[1].loc.lat)
    
    sort_method = sort([corr.loc.lat for corr in corrs])
#         sort_method = sort([corr.latitude for corr in corrs])#sort([get_dist(source_geo, corr.loc) for corr in corrs])
    p = sortperm([corr.loc.lat for corr in corrs])#[get_dist(source_geo, corr.loc) for corr in corrs])
        #println("Not currently a supported stacktype")


    lmin, lmax = minimum(sort_method), maximum(sort_method)
    new_corrs, n_corrs = corrs[p], length(corrs)
    y_labels = (round.(collect(range(0, 10*n_corrs, length=10)),digits=3),round.(collect(range(lmin, lmax, length=10)),digits=3))
    for freq_pair in freqs
        NS_plot = plot(xlims = (-100,100), ylims = (-10,n_corrs*10+15), 
                    yticks=y_labels,xlabel = "Time (s)", ylabel = "Station Nodes by $plot_sorttype (lat)",  
                    xtickfontsize=5,ytickfontsize=5,fontsize=5, xguidefontsize = 10, yguidefontsize = 10,
                    legendfontsize = 10, title="$(attr[3]) $(attr[2]) from $(attr[1]): $(freq_pair[1]) to $(freq_pair[2])",
                    titlefontsize= 10)
        # bandpass correlations 
        corrs_processed = []
        max_amp = []
        ys, maxloc = [], []
        for (ind, corr) in enumerate(new_corrs)
            try
                # bandpass, scale, shift, and add to plot
                dist = corr.dist #ind*5#get_dist(source_geo, corr.loc)
                taper!(corr)
                processed = bandpass(fftderivative(corr).corr, freq_pair[1], freq_pair[2], 20.)
                nwin2 = convert(Int64, round(length(processed)/2))
                t2=convert(Int64, floor(nwin2-(dist/5)*corr.fs))
                t1=convert(Int64, ceil(nwin2-(dist/1.5)*corr.fs))
                trange = collect(t1:1:t2) # anticausal window of ballistic waves
                imax = findmax(processed[trange])[2]+t1
                push!(maxloc, imax)
                push!(ys, ind*10)
                push!(max_amp, findmax(processed[trange]))
                push!(corrs_processed, processed./findmax(processed)[1].*5 )
            catch e
                println(e)
            end
        end
        # get the max amp
        mmax = findmax(max_amp)[1][1]
        T = collect(-300.:1/fs:300.)
        crap=[]
        for i in collect(1:length(corrs_processed))
            push!(crap,corrs_processed[i]./max_amp[i][1].*mmax .+ 10*i)

        end
        Plots.plot!(NS_plot, T, crap, color=:cm_maxamp,  
                line_z=getindex.(max_amp,1)', fmt = :png, linewidth = 0.5, reuse = false, legend = false)  
#         plot!(NS_plot, T, crap, color=:cm_maxamp, colorbar_title="Normalized Maximum Amplitude", 
#                 line_z=getindex.(max_amp,1)', fmt = :png, linewidth = 0.5, reuse = false, legend = false)     
        Plots.plot!(size=(350,400),dpi=500)
        
        # add dots where max_amp is chosen 
        dot_locations = [amp[2] for amp in max_amp]
        y = 10 .* collect(0:length(max_amp))
        mloc = [T[ind] for ind in maxloc]
        scatter!(mloc, ys, color="black", markersize=1.2)
        filepath = joinpath("/Users/marinedenolle/BASIN/nodestack_plots/$(attr[1])/")
        DIR = dirname(filepath)
        if !isdir(DIR)
            mkpath(DIR)
        end
        print(filepath)
        png(NS_plot,filepath*"$(new_corrs[1].name[1:5])_waveforms_$(attr[2])_$(freq_pair[1])to$(freq_pair[2])_$(attr[4]).png")
    end
end

plot_by_latitude (generic function with 4 methods)

In [15]:
?fftderivative

search: fftderivative



No documentation found.

`fftderivative` is a `Function`.

```
# 1 method for generic function "fftderivative":
[1] fftderivative(A::CorrData) in Main at /Users/marinedenolle/BASIN/C4-Project.jl/src/process.jl:5
```


In [16]:
# metaparams for plots
n_derivatives = 2
n_deriv_names = ["Raw","PGD","PDV"]
#plot_sorttype = "Latitude" # other options include "Longitude" and (soon) "Distance" (in progress...) 
years = [2017]#, 2018, 2019]
components = ["ZZ","EE","EN","EZ","NE", "NN", "NZ","ZE","ZN", "ZZ"]
source_stations = ["SVD"]#["CHN","CJM","DEV","IPT","LPC","SNO","SVD","TA2"]
rootdir = "/Users/marinedenolle/BASIN" # chose root directory for plots
lw, scale, frequency_plots = 0.5, 3, [[0.1,0.35]] #  [[0.1,0.2],[0.15, 0.25],[0.2,0.3],[0.1,0.35]]
locations = DataFrame(CSV.File("/Users/marinedenolle/BASIN/C4-Project.jl/docs/modified_nodal.csv"));

In [17]:
files = glob("nodestack/*",rootdir);
println(files)
filter!(x -> occursin("SVD", x), files)

["/Users/marinedenolle/BASIN/nodestack/2017", "/Users/marinedenolle/BASIN/nodestack/2018", "/Users/marinedenolle/BASIN/nodestack/2019", "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_allday.h5", "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_ref.h5", "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_rmamp.h5", "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_rmamp_night.h5", "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_rmamp_night_shortwindows.h5", "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_strict.h5"]


6-element Vector{String}:
 "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_allday.h5"
 "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_ref.h5"
 "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_rmamp.h5"
 "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_rmamp_night.h5"
 "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_rmamp_night_shortwindows.h5"
 "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_strict.h5"

In [25]:
# print(files)
f=files[3]
println(typeof(f))
year = split(split(split(f,"/")[end],"_")[end-1],".")[1]
println(year)
println(f)
processingtype = split(split(split(f,"/")[end],"_")[end],".")[1]
println(processingtype)

String
2017
/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_rmamp.h5
rmamp


## Stackplot Script for BASIN Data

In [26]:
plot_sorttype = "Latitude"
println(f)
# year = split(split(split(f,"/")[end],"_")[end-1],".")[1]
# year = split(split(split(f,"/")[end],"_")[end-1],".")[1]
println(year)
# 
try  
    file = h5open(f,"r")
    print(f)
    for comp in components
        for (ind, sktype) in enumerate(["linear"])#,"pws","robust"])
            for filt in ["NO.B4"]#,"NO.B2","NO.B3", "NO.B4","NO.B5","NO.B6","NO.G1","NO.G2","NO.G3","NO.G4"]
                source, stacktype, component, filter = convert(String, split(f, ".")[end-1][1:3]), sktype, comp, filt
                println( source, stacktype, component, filter )
                fcorrs = get_corrs(file, stacktype, component, filter)
#                 println(fcorrs[1].corr)
                println(length(fcorrs))
                if length(fcorrs) > 1
#                         println("Plotting $filt $comp $sktype for $source in $year")
                    attributes = [source,component,stacktype,processingtype]
                    plot_by_latitude(fcorrs, frequency_plots,attributes)
#                         println("finised plotting")
                end
            end
        end
    end
catch e
    println(e)
end

/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_rmamp.h5
2017
/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_rmamp.h5SVDlinearZZNO.B4
90
MethodError(plot_by_latitude, (CorrData[CorrData with 0 Corrs
      NAME: "NO.B4003"                         
        ID: ""                                 
       LOC: 34.1583 N, -117.647 E, 649.0 m
      COMP: ""                                 
   ROTATED: false                              
 CORR_TYPE: ""                                 
        FS: 20.0
      GAIN: 1.0
   FREQMIN: 0.0
   FREQMAX: 0.0
    CC_LEN: 0.0
   CC_STEP: 0.0
  WHITENED: false                              
 TIME_NORM: ""                                 
      RESP: a0 1.0, f0 1.0, 0z, 0p
      MISC: 0 entries                          
     NOTES: 0 entries                          
      DIST: 50.9404
       AZI: 276.638
       BAZ: 96.3301
    MAXLAG: 0.0
         T: Float64[]                          
      CORR: 12001×1 Matrix{Float64}            
, CorrData with 0 C

  WHITENED: false                              
 TIME_NORM: ""                                 
      RESP: a0 1.0, f0 1.0, 0z, 0p
      MISC: 0 entries                          
     NOTES: 0 entries                          
      DIST: 50.7251
       AZI: 272.686
       BAZ: 92.3777
    MAXLAG: 0.0
         T: Float64[]                          
      CORR: 12001×1 Matrix{Float64}            
, CorrData with 0 Corrs
      NAME: "NO.B4016"                         
        ID: ""                                 
       LOC: 34.1247 N, -117.647 E, 462.0 m
      COMP: ""                                 
   ROTATED: false                              
 CORR_TYPE: ""                                 
        FS: 20.0
      GAIN: 1.0
   FREQMIN: 0.0
   FREQMAX: 0.0
    CC_LEN: 0.0
   CC_STEP: 0.0
  WHITENED: false                              
 TIME_NORM: ""                                 
      RESP: a0 1.0, f0 1.0, 0z, 0p
      MISC: 0 entries                          
     NOTES: 0 entr

   ROTATED: false                              
 CORR_TYPE: ""                                 
        FS: 20.0
      GAIN: 1.0
   FREQMIN: 0.0
   FREQMAX: 0.0
    CC_LEN: 0.0
   CC_STEP: 0.0
  WHITENED: false                              
 TIME_NORM: ""                                 
      RESP: a0 1.0, f0 1.0, 0z, 0p
      MISC: 0 entries                          
     NOTES: 0 entries                          
      DIST: 50.6849
       AZI: 267.635
       BAZ: 87.3275
    MAXLAG: 0.0
         T: Float64[]                          
      CORR: 12001×1 Matrix{Float64}            
, CorrData with 0 Corrs
      NAME: "NO.B4031"                         
        ID: ""                                 
       LOC: 34.0842 N, -117.647 E, 347.0 m
      COMP: ""                                 
   ROTATED: false                              
 CORR_TYPE: ""                                 
        FS: 20.0
      GAIN: 1.0
   FREQMIN: 0.0
   FREQMAX: 0.0
    CC_LEN: 0.0
   CC_STEP: 0.0
  WH

      NAME: "NO.B4043"                         
        ID: ""                                 
       LOC: 34.0514 N, -117.647 E, 287.0 m
      COMP: ""                                 
   ROTATED: false                              
 CORR_TYPE: ""                                 
        FS: 20.0
      GAIN: 1.0
   FREQMIN: 0.0
   FREQMAX: 0.0
    CC_LEN: 0.0
   CC_STEP: 0.0
  WHITENED: false                              
 TIME_NORM: ""                                 
      RESP: a0 1.0, f0 1.0, 0z, 0p
      MISC: 0 entries                          
     NOTES: 0 entries                          
      DIST: 51.0649
       AZI: 263.282
       BAZ: 82.9745
    MAXLAG: 0.0
         T: Float64[]                          
      CORR: 12001×1 Matrix{Float64}            
, CorrData with 0 Corrs
      NAME: "NO.B4044"                         
        ID: ""                                 
       LOC: 34.0494 N, -117.647 E, 282.0 m
      COMP: ""                                 
   ROTATED

       AZI: 259.107
       BAZ: 78.8007
    MAXLAG: 0.0
         T: Float64[]                          
      CORR: 12001×1 Matrix{Float64}            
, CorrData with 0 Corrs
      NAME: "NO.B4057"                         
        ID: ""                                 
       LOC: 34.0169 N, -117.651 E, 238.0 m
      COMP: ""                                 
   ROTATED: false                              
 CORR_TYPE: ""                                 
        FS: 20.0
      GAIN: 1.0
   FREQMIN: 0.0
   FREQMAX: 0.0
    CC_LEN: 0.0
   CC_STEP: 0.0
  WHITENED: false                              
 TIME_NORM: ""                                 
      RESP: a0 1.0, f0 1.0, 0z, 0p
      MISC: 0 entries                          
     NOTES: 0 entries                          
      DIST: 51.9489
       AZI: 259.134
       BAZ: 78.8244
    MAXLAG: 0.0
         T: Float64[]                          
      CORR: 12001×1 Matrix{Float64}            
, CorrData with 0 Corrs
      NAME: "NO.B4058

 TIME_NORM: ""                                 
      RESP: a0 1.0, f0 1.0, 0z, 0p
      MISC: 0 entries                          
     NOTES: 0 entries                          
      DIST: 52.5471
       AZI: 255.029
       BAZ: 74.721
    MAXLAG: 0.0
         T: Float64[]                          
      CORR: 12001×1 Matrix{Float64}            
, CorrData with 0 Corrs
      NAME: "NO.B4070"                         
        ID: ""                                 
       LOC: 33.9809 N, -117.648 E, 192.0 m
      COMP: ""                                 
   ROTATED: false                              
 CORR_TYPE: ""                                 
        FS: 20.0
      GAIN: 1.0
   FREQMIN: 0.0
   FREQMAX: 0.0
    CC_LEN: 0.0
   CC_STEP: 0.0
  WHITENED: false                              
 TIME_NORM: ""                                 
      RESP: a0 1.0, f0 1.0, 0z, 0p
      MISC: 0 entries                          
     NOTES: 0 entries                          
      DIST: 52.6022

 CORR_TYPE: ""                                 
        FS: 20.0
      GAIN: 1.0
   FREQMIN: 0.0
   FREQMAX: 0.0
    CC_LEN: 0.0
   CC_STEP: 0.0
  WHITENED: false                              
 TIME_NORM: ""                                 
      RESP: a0 1.0, f0 1.0, 0z, 0p
      MISC: 0 entries                          
     NOTES: 0 entries                          
      DIST: 53.7409
       AZI: 250.74
       BAZ: 70.4326
    MAXLAG: 0.0
         T: Float64[]                          
      CORR: 12001×1 Matrix{Float64}            
, CorrData with 0 Corrs
      NAME: "NO.B4084"                         
        ID: ""                                 
       LOC: 33.9432 N, -117.646 E, 161.0 m
      COMP: ""                                 
   ROTATED: false                              
 CORR_TYPE: ""                                 
        FS: 20.0
      GAIN: 1.0
   FREQMIN: 0.0
   FREQMAX: 0.0
    CC_LEN: 0.0
   CC_STEP: 0.0
  WHITENED: false                              
 TIME

        ID: ""                                 
       LOC: 33.9102 N, -117.649 E, 156.0 m
      COMP: ""                                 
   ROTATED: false                              
 CORR_TYPE: ""                                 
        FS: 20.0
      GAIN: 1.0
   FREQMIN: 0.0
   FREQMAX: 0.0
    CC_LEN: 0.0
   CC_STEP: 0.0
  WHITENED: false                              
 TIME_NORM: ""                                 
      RESP: a0 1.0, f0 1.0, 0z, 0p
      MISC: 0 entries                          
     NOTES: 0 entries                          
      DIST: 55.3327
       AZI: 246.983
       BAZ: 66.6755
    MAXLAG: 0.0
         T: Float64[]                          
      CORR: 12001×1 Matrix{Float64}            
], [[0.1, 0.35]], AbstractString["SVD", "ZZ", "linear", "rmamp"]), 0x0000000000007b7a)
